In [1]:
import os
import pandas as pd
import numpy as np
[_ for _ in os.listdir('./') if '2023-10-01' in _ and _.endswith('.zip')]

['BTCUSDT-aggTrades-2023-10-01.zip', 'BTCUSDT-bookTicker-2023-10-01.zip']

In [2]:
pd.read_csv('./BTCUSDT-bookTicker-2023-10-01.zip', nrows=1000)

,update_id,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty,transaction_time,event_time
0,3316349889389,26950.9,8.285,26951.0,15.869,1696118400003,1696118400008
1,3316349889487,26950.9,8.285,26951.0,15.862,1696118400006,1696118400012
2,3316349889615,26950.9,8.285,26951.0,15.855,1696118400010,1696118400015
3,3316349889994,26950.9,8.285,26951.0,15.911,1696118400019,1696118400024
4,3316349891202,26950.9,8.218,26951.0,15.911,1696118400035,1696118400040
...,...,...,...,...,...,...,...
995,3316352305317,26946.4,9.829,26946.5,4.261,1696118409542,1696118409547
996,3316352305633,26946.4,9.836,26946.5,4.261,1696118409549,1696118409554
997,3316352305811,26946.4,9.843,26946.5,4.261,1696118409553,1696118409558
998,3316352306160,26946.4,9.836,26946.5,4.261,1696118409561,1696118409565


In [2]:
bookTicker = (
    pd.read_csv(
        "./BTCUSDT-bookTicker-2023-10-01.zip",
        # nrows=10000,
        usecols=[
            "transaction_time",
            "best_bid_price",
            "best_bid_qty",
            "best_ask_price",
            "best_ask_qty",
            "update_id",
        ],
    )
    .sort_values(["transaction_time", "update_id"])
    .drop_duplicates(["transaction_time"], keep="last", ignore_index=True)
)
del bookTicker["update_id"]
# bookTicker.columns = ['bp', 'bv', 'ap', 'av', 'ts']
bookTicker.rename(
    columns={
        "transaction_time": "ts",
        "best_bid_price": "bp",
        "best_bid_qty": "bv",
        "best_ask_price": "ap",
        "best_ask_qty": "av",
    },
    inplace=True,
)
bookTicker = bookTicker[["ts", "bp", "bv", "ap", "av"]]
bookTicker.dtypes

ts      int64
bp    float64
bv    float64
ap    float64
av    float64
dtype: object

In [3]:
bookTicker.head()

,ts,bp,bv,ap,av
0,1696118400003,26950.9,8.285,26951.0,15.869
1,1696118400006,26950.9,8.285,26951.0,15.862
2,1696118400010,26950.9,8.285,26951.0,15.855
3,1696118400019,26950.9,8.285,26951.0,15.911
4,1696118400035,26950.9,8.218,26951.0,15.911


In [4]:
aggTrades = pd.read_csv(
    "./BTCUSDT-aggTrades-2023-10-01.zip",
    # nrows=10000,
    usecols=["price", "quantity", "transact_time", "is_buyer_maker", "agg_trade_id"],
).sort_values(["transact_time", "agg_trade_id"])
del aggTrades["agg_trade_id"]
aggTrades.rename(columns={"price": "p", "quantity": "v", "transact_time": "ts"}, inplace=True)

In [5]:
aggTrades.dtypes

p                 float64
v                 float64
ts                  int64
is_buyer_maker       bool
dtype: object

In [6]:
aggTrades.head()

,p,v,ts,is_buyer_maker
0,26951.0,0.003,1696118404732,False
1,26950.9,0.005,1696118404758,True
2,26951.0,0.001,1696118404760,False
3,26950.9,0.023,1696118404769,True
4,26951.0,0.001,1696118404771,False


In [7]:
aggTrades['v'] = np.where(aggTrades.is_buyer_maker, -aggTrades.v, aggTrades.v)
del aggTrades['is_buyer_maker']
aggTrades = aggTrades[['ts', 'p', 'v']]
aggTrades.head()

,ts,p,v
0,1696118404732,26951.0,0.003
1,1696118404758,26950.9,-0.005
2,1696118404760,26951.0,0.001
3,1696118404769,26950.9,-0.023
4,1696118404771,26951.0,0.001


In [8]:
unique_trades_ts_ = aggTrades.ts.drop_duplicates()
ticker = pd.merge(unique_trades_ts_, bookTicker, how='outer', on='ts')
ticker.ffill(inplace=True)
ticker['mp'] = (ticker.bp + ticker.ap) / 2

In [9]:
ticker

,ts,bp,bv,ap,av,mp
0,1696118400003,26950.9,8.285,26951.0,15.869,26950.95
1,1696118400006,26950.9,8.285,26951.0,15.862,26950.95
2,1696118400010,26950.9,8.285,26951.0,15.855,26950.95
3,1696118400019,26950.9,8.285,26951.0,15.911,26950.95
4,1696118400035,26950.9,8.218,26951.0,15.911,26950.95
...,...,...,...,...,...,...
5540991,1696204799942,27981.3,12.309,27981.4,7.393,27981.35
5540992,1696204799950,27981.3,12.309,27981.4,7.393,27981.35
5540993,1696204799952,27981.3,12.309,27981.4,7.368,27981.35
5540994,1696204799958,27981.3,12.309,27981.4,7.258,27981.35


In [10]:
trades = pd.merge_asof(aggTrades, ticker[['ts', 'mp']], on='ts', allow_exact_matches=False)

In [11]:
trades

,ts,p,v,mp
0,1696118404732,26951.0,0.003,26950.95
1,1696118404758,26950.9,-0.005,26950.95
2,1696118404760,26951.0,0.001,26950.95
3,1696118404769,26950.9,-0.023,26950.95
4,1696118404771,26951.0,0.001,26950.95
...,...,...,...,...
758280,1696204799665,27981.4,0.001,27981.35
758281,1696204799813,27981.3,-0.005,27981.35
758282,1696204799822,27981.4,0.160,27981.35
758283,1696204799927,27981.3,-1.000,27981.35


In [12]:
ticker.to_parquet('./BTCUSDT-bookTicker-2023-10-01.p')
trades.to_parquet('./BTCUSDT-aggTrades-2023-10-01.p')